In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pmdarima.arima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Function to automatically select and fit SARIMA model
def run_auto_sarima(series_i):
    """Runs a grid search for the best SARIMA model parameters and fits the SARIMA model."""
    gridsearch = auto_arima(
        series_i,
        start_p=0, max_p=3,
        d=None, max_d=3,
        start_q=0, max_q=3,
        seasonal=True,
        m=12,  # Monthly data seasonality
        start_P=0, max_P=3,
        start_Q=0, max_Q=3,
        max_order=10,
        stepwise=True,
        suppress_warnings=True
    )
    
    model = SARIMAX(
        series_i,
        order=gridsearch.order,
        seasonal_order=gridsearch.seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    return model.fit()

# Function to execute SARIMA model and forecast
def run_sarima_model(i, steps, df):
    """Runs SARIMA on the selected time series, returning forecast results."""
    series = df.iloc[:, i:i+1]
    name = series.columns[0]
    log_series = np.log(series).dropna()
    
    model = run_auto_sarima(log_series)
    log_forecast = model.get_forecast(steps)
    forecast_series = np.exp(log_forecast.summary_frame())
    
    return name, series, forecast_series

# Function to plot results
def plot_sarima_results(i, steps, df):
    name, original_series, forecast_series = run_sarima_model(i, steps, df)
    
    plt.figure(figsize=(15, 7))
    plt.plot(original_series, label='Original')
    plt.plot(forecast_series['mean'], label='Predicted')
    plt.fill_between(
        forecast_series.index, 
        forecast_series['mean_ci_lower'], 
        forecast_series['mean_ci_upper'], 
        color='gray', alpha=0.2
    )
    plt.title(name)
    plt.legend()
    plt.xlabel('Year')
    plt.ylabel('Median Home Price')
    plt.show()
    
    forecast = round(forecast_series['mean'][steps - 1])
    low_int = round(forecast_series['mean_ci_lower'][steps - 1])
    high_int = round(forecast_series['mean_ci_upper'][steps - 1])
    print(f"{steps}-month forecast: {forecast}")
    print(f"95% confidence interval: {low_int} - {high_int}")

# Example usage (Make sure df_2018 is loaded before running this)
# plot_sarima_results(1, 12, df_2018)
